In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from env.EV_Sce_Env import EV_Sce_Env
import math
import seaborn as sns
import numpy as np

env = EV_Sce_Env('HY_2', seed=0)

In [13]:
point = gpd.read_file('shp/YH.shp')
link = gpd.read_file('shp/YH_link.shp')

agent_num = env.agent_num
od_id = {}
for i in range(link.shape[0]):
    od_id[link.iloc[i]['o-d']] = i

Route Choice

In [ ]:
link_list = list(set(link['o-d']))
link_dic = {}
for l in link_list:
    link_dic[l] = 0

for i in range(agent_num):
    route = pd.read_csv('output/HY_2_T1_MAPPO/EV/EV%i_route.csv' % i)
    for j in range(route.shape[0]):
        link_dic[route.iloc[j]['edge']] += 1

link_count = pd.DataFrame(list(link_dic.items()), columns=['o-d', 'count'])
link1 = pd.merge(link, link_count, how='left', on='o-d')
link1['count'] = link1['count'].fillna(0)

link1.to_file('shp/YH_link_test.shp', driver='ESRI Shapefile')

Route Couonts

In [5]:
route_con = {}
route_list = {}
for i in range(env.agent_num):
    ev_id = i
    df_r = pd.read_csv('output/HY_2_T1_MAPPO/EV/EV{}_route.csv'.format(ev_id))
    route = list(df_r['edge'])
    r = route[0]
    for j in range(1, len(route)):
        next_p = route[j].split('-')[-1]
        r += "-"+next_p
    if r not in route_con.keys():
        route_con[r] = 1
        route_list[r] = [ev_id]
    else:
        route_con[r] += 1
        route_list[r].append(ev_id)

In [33]:
link2 = link.copy()
for r in route_con.keys():
    link2[r] = 0
for r in route_con.keys():
    point_seq = r.split('-')
    for i in range(1, len(point_seq)):
        link2.loc[od_id[point_seq[i-1]+'-'+point_seq[i]], r] = route_con[r]

link2.to_file('shp/YH_link_test2.shp', driver='ESRI Shapefile')

C:\Users\10199\AppData\Local\Temp\ipykernel_23264\902422355.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  link2.to_file('shp/YH_link_test2.shp', driver='ESRI Shapefile')


Route Attribute

In [36]:
route_list

{'0-1-2-3-4-9': [0, 1, 3, 4, 7, 9, 10, 14, 16, 17, 19, 21, 28],
 '0-6-9': [2, 5, 27],
 '0-6-7-8-9': [6, 8, 15, 18, 23, 25, 26],
 '0-1-2-3-4-5-9': [11, 12, 22, 24],
 '0-2-3-4-5-9': [13],
 '0-1-2-3-9': [20, 29]}

Charging time at CS